# colab을 이용한 기계학습 실습 pt.2

🎯 학습 목표 : colab 환경에서 자연어 데이터를 활용하는 기계 학습 코드를 실행하고 결과를 확인할 수 있다.

- 실습 재료

| 항목 | 상세 |
| ---- | ---- |
| 🗂️ 데이터 | AI-HUB의 감성 대화 말뭉치 |
| 🤖 기계 학습 알고리즘 | BERT |
| 🏗️ 기계 학습 프레임워크 | transformers |
| 🐍 프로그래밍 언어 | Python |
| 👩‍💻 프로그래밍 환경 | Colab |


- colab에서 코드 실행 방법은 다음 그림을 참조해주시기 바랍니다.
    ![](https://i.imgur.com/0GoFr7q.png)

# **주의** 본 실습은 GPU를 사용합니다. colab 런타임에 GPU 를 사용하도록 변경하여야 합니다!

![](https://i.imgur.com/JrUzpgk.png)

<!-- ## 1. Colab이란?

- colab은 구글에서 제공하는 온라인 기계 학습 환경 입니다.

![](https://post-phinf.pstatic.net/MjAxOTEwMTVfMTE4/MDAxNTcxMTA3ODE4NTcy.KmzXuRSS3HWe4qnBR7esUkTOCPELkbi6fD0khAX8i8kg.9KgltPv7JsznlhiQVmmCxwVFBqLUI03VZaAwFwxUfHkg.JPEG/29.JPG?type=w1200)
*(그림 출처 : [네이버 블로그](https://post.naver.com/viewer/postView.nhn?volumeNo=26447765))*

- 기계 학습 코드를 작성하고, 실행할 수 있습니다.
  - 기계 학습 코드를 작성하기 위해서 필요한 것은 웹 브라우저뿐 입니다! 지금 여러분들이 보고 계시듯이 말이죠.
  - 실행은 구글의 데이터 센터에서 실행됩니다. -->

### 1. 필요 라이브러리 및 함수 추가

실습에 앞서 아래의 코드 셀을 실행해주시기 바랍니다.

In [ ]:
# 이번 실습에 사용하는 데이터 및 함수를 다운로드 받습니다.
## 참고 : 느낌표(!)로 시작하는 코드는 python이 아닌, linux 운영체제 명령어로 실행됩니다.
!rm ai_practice -rf
!git clone https://github.com/taeminlee/ai_practice

이번 실습에서는 딥러닝을 이용하여 모델을 만듭니다.

다음의 코드 셀을 실행하여 딥러닝 과정에서 사용하는 라이브러리를 설치합시다.

In [ ]:
!pip install transformers datasets

### 2. 실습 목표 및 데이터 살펴보기

이번 실습에서 사용되는 데이터는 [AI-HUB의 감성 대화 말뭉치](https://www.aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=realm&dataSetSn=86) 입니다.

해당 데이터를 이용하여 `사용자의 감정이 섞인 문장(text)`이 주어졌을 때, 어떠한 `감정`인지 판별(classifiaction)하는 모델을 만들어보도록 하겠습니다.

![](https://i.imgur.com/s0Wa5WT.png)

실습 데이터 집합에 대한 명세는 다음과 같습니다.

- 데이터 개수 : 학습 데이터 51,360건, 검증 데이터 6,641건
- 사용 항목
  - 사람문장1
    - 예) `일은 왜 해도 해도 끝이 없을까? 화가 난다.`
  - 감정_대분류
    - 예) `분노`
    - 감정 클래스 : ('분노', '기쁨', '불안', '당황', '슬픔', '상처')

In [ ]:
# 데이터를 불러옵니다.
## 참고 : 해당 데이터 처리 코드는 https://github.com/taeminlee/ai_practice/data_util_2.py 에 기록되어 있습니다.
from ai_practice.data_util_2 import dataset, sentiments

In [ ]:
# 학습 데이터를 살펴보기 위해 이 코드 블록을 실행시켜 주십시오.
dataset['train'][0]

In [ ]:
# 검증 데이터를 살펴보기 위해 이 코드 블록을 실행시켜 주십시오.
dataset['test'][0]

### 3. 데이터 전처리

자연어로 기술된 데이터를 딥러닝을 이용해 분석하기 위해서는 수치 값으로 변환이 필요합니다.

이 중 가장 중요한 것은 [문장을 토큰으로 변환](https://wikidocs.net/21698)하는 과정입니다.

다음의 코드 셀을 실행해서 토큰화 과정을 수행합니다.

In [ ]:
# 토큰화 과정에서 필요한 라이브러리를 준비합니다.
from transformers import AutoTokenizer, DataCollatorWithPadding
import torch

# 토큰화
tokenizer = AutoTokenizer.from_pretrained('beomi/kcbert-base')

def preprocess_function(examples):
    tokenized_examples = tokenizer(str(examples["사람문장1"]))
    
    return tokenized_examples

tokenized_dataset = dataset.map(preprocess_function)
tokenized_dataset.set_format(type='torch', columns=['input_ids', 'labels', 'attention_mask', 'token_type_ids'])

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
# 토큰화 결과를 확인하기 위해 코드 블록을 실행해 주시기 바랍니다.
tokenized_dataset['train'][0]

### 4. 모델 학습

본 실습에서는 딥러닝 알고리즘 중 transfomer 계열의 BERT 네트워크를 기반으로 만들어진 KcBERT를 미세조정(fine tuning)하여 모델을 학습할 것 입니다.

학습을 위해 [transformers](https://huggingface.co/docs/transformers/index) 라이브러리를 이용합니다.

본 실습에서는 BERT구조로 모델을 학습할 것이므로, transformer 중 [BERT](https://huggingface.co/docs/transformers/model_doc/bert) 를 사용합니다.

*(참고) 그 외 네트워크들은 [transformers 홈페이지의 summary of the models 페이지](https://huggingface.co/docs/transformers/model_summary) 에서 확인 가능합니다.*

transformers 라이브러리에서 학습은 `Trainer` 클래스를 이용하여 수행하도록 구성되어 있습니다.

다음의 코드는 KcBERT 기반 감성 클래스 분류기를 생성하고, 학습하는 과정입니다.

In [ ]:
# transformers 라이브러리에서 BERT 분류기 클래스를 불러옵니다.
from transformers import BertForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import label_ranking_average_precision_score


# 감성 분류 BERT 모델 객체를 생성합니다.
model = BertForSequenceClassification.from_pretrained(
    'beomi/kcbert-base', 
    num_labels=6, 
    problem_type="multi_label_classification"
)
model.config.id2label = {i: label for i, label in zip(range(6), sentiments)}
model.config.label2id = {label: i for i, label in zip(range(6), sentiments)}
#
model.to('cuda')

def compute_metrics(x):
    return {
        'lrap': label_ranking_average_precision_score(x.label_ids, x.predictions),
    }

args = TrainingArguments(
    output_dir="model_output",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    num_train_epochs=2,
    save_strategy='epoch',
    load_best_model_at_end=True,
    metric_for_best_model='lrap',
    greater_is_better=True,
)

trainer = Trainer(
    model=model, 
    args=args, 
    train_dataset=tokenized_dataset["train"], 
    eval_dataset=tokenized_dataset["test"], 
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    data_collator=data_collator
)

trainer.train()
# 결과 모델은 trainer 변수에 저장되어 있습니다.

다음의 코드는 학습된 모델에 임의의 문장을 넣어 테스트 하는 코드입니다.

In [ ]:
from transformers import TextClassificationPipeline

pipe = TextClassificationPipeline(
    model = model,
    tokenizer = tokenizer,
    device=0,
    return_all_scores=True,
    function_to_apply='sigmoid'
    )

In [ ]:
example_text = "야 이 개새끼들아!! 개 짖는 소리좀 안나게 하그라!"  # @param
for result in pipe(example_text)[0]:
    print(result)

이상으로 본 실습을 마치도록 하겠습니다.

모두들 고생 많으셨습니다!

![](https://img.favpng.com/10/1/7/kakaotalk-kakao-friends-emoticon-sticker-png-favpng-mZm2vp0mk2Ce9aTUnBjC4s4DZ.jpg)